# 真實世界的機器學習範例
* 前半段使用已知函數來調整股價，因此知道哪些因子對股價影響較大。但現實世界並非如此！

In [3]:
import pandas as pd

def generate_ml_features(df, window_short=5, window_long=20):
    """
    根據單支股票歷史 K 線資料，生成 ML 特徵
    
    df 必須包含：
        ['Open', 'High', 'Low', 'Close', 'Volume']
    """

    df = df.copy()

    # --- 1️⃣ Price Rank (相對位置) ---
    df["price_rank_long"] = df["Close"].rank(pct=True)
    df["price_rank_short"] = df["Close"].rolling(window_short).rank(pct=True)

    # --- 2️⃣ 動能角度（近 N 日斜率）---
    df["price_slope"] = (
        df["Close"].diff(window_short) / window_short
    )

    # --- 3️⃣ 波動度（真實波幅 ATR）---
    high_low = df["High"] - df["Low"]
    high_close = (df["High"] - df["Close"].shift()).abs()
    low_close = (df["Low"] - df["Close"].shift()).abs()
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)

    df["atr"] = tr.rolling(window_long).mean()

    # --- 4️⃣ 波動標準差 ---
    df["volatility"] = df["Close"].pct_change().rolling(window_long).std()

    # --- 5️⃣ 跳空缺口 ---
    df["gap"] = (df["Open"] - df["Close"].shift()) / df["Close"].shift()

    # --- 6️⃣ 成交量變化 ---
    df["volume_change"] = df["Volume"].pct_change()

    # --- 7️⃣ 隔日報酬（label 用）---
    df["target_return"] = df["Close"].pct_change().shift(-1)

    # --- 刪除前期無效值 ---
    df = df.dropna()

    return df